In [ ]:
# 将

In [6]:
import os
import shutil
import csv

# 定义文件路径
forseg_path = "../Raw/ForSeg"  # 调整到正确的相对路径
images_path = os.path.join("../Processed/PII_Full", "images")
labels_path = os.path.join("../Processed/PII_Full", "labels")
csv_file = os.path.join("../Processed/PII_Full", "PII_log.csv")

# 检查ForSeg路径是否存在
if not os.path.exists(forseg_path):
    raise FileNotFoundError(f"路径不存在: {forseg_path}")

# 如果images和labels文件夹不存在则创建
os.makedirs(images_path, exist_ok=True)
os.makedirs(labels_path, exist_ok=True)

# 初始化全局的文件编号计数
global_count = 1

# 打开CSV文件，准备记录
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    # 写入CSV文件的头部
    writer.writerow(["Original MRI File", "Original Seg File", "Dataset", "New Filename"])
    
    # 遍历ForSeg下的所有文件夹
    for folder_name in os.listdir(forseg_path):
        folder_path = os.path.join(forseg_path, folder_name)
        
        # 如果不是文件夹，跳过
        if not os.path.isdir(folder_path):
            continue
        
        # 确保文件夹命名符合PII_NN_WKXX格式
        if not folder_name.startswith("PII_") or "_WK" not in folder_name:
            continue
        
        # 定位b0_img.nii和seg.nii文件
        b0_img_file = os.path.join(folder_path, "b0_img.nii")
        seg_file = os.path.join(folder_path, "seg.nii")
        
        # 如果b0_img.nii和seg.nii都存在，则进行复制
        if os.path.exists(b0_img_file) and os.path.exists(seg_file):
            # 构造全局递增的文件名
            new_file_name = f"PII_{str(global_count).zfill(3)}.nii"
            
            # 复制b0_img.nii到images文件夹
            new_b0_img_path = os.path.join(images_path, new_file_name)
            shutil.copy2(b0_img_file, new_b0_img_path)
            
            # 复制seg.nii到labels文件夹
            new_seg_path = os.path.join(labels_path, new_file_name)
            shutil.copy2(seg_file, new_seg_path)
            
            # 记录到CSV文件
            writer.writerow([b0_img_file, seg_file, "images", new_file_name])
            writer.writerow([b0_img_file, seg_file, "labels", new_file_name])
            
            # 递增全局计数
            global_count += 1
        else:
            print(f"Warning: Missing files in {folder_name}")

print("Task completed! File copies and CSV log created.")

Task completed! File copies and CSV log created.


In [7]:
import os
import shutil
import random
import csv

# 定义PII_Full路径
images_full_path = os.path.join("../Processed/PII_Full", "images")
labels_full_path = os.path.join("../Processed/PII_Full", "labels")

# 定义分割后文件夹路径
split_base_path = "../Processed/PII_Splite"
images_tr_path = os.path.join(split_base_path, "imagesTr")
labels_tr_path = os.path.join(split_base_path, "labelsTr")
images_ts_path = os.path.join(split_base_path, "imagesTs")
labels_ts_path = os.path.join(split_base_path, "labelsTs")

# 创建Splite下的文件夹
os.makedirs(images_tr_path, exist_ok=True)
os.makedirs(labels_tr_path, exist_ok=True)
os.makedirs(images_ts_path, exist_ok=True)
os.makedirs(labels_ts_path, exist_ok=True)

# 获取所有文件名（假设images和labels中的文件名相同）
all_files = os.listdir(images_full_path)

# 随机打乱文件顺序
random.shuffle(all_files)

# 按2:8比例分割训练集和测试集
split_index = int(0.8 * len(all_files))
train_files = all_files[:split_index]
test_files = all_files[split_index:]

# 创建CSV日志文件
train_csv = os.path.join(split_base_path, "train_log.csv")
test_csv = os.path.join(split_base_path, "test_log.csv")

# 处理训练集
with open(train_csv, mode='w', newline='') as train_file:
    train_writer = csv.writer(train_file)
    train_writer.writerow(["Original MRI Path", "Original Seg Path", "New MRI Path", "New Seg Path"])
    
    for filename in train_files:
        # 定义原始路径
        original_img = os.path.join(images_full_path, filename)
        original_seg = os.path.join(labels_full_path, filename)
        
        # 定义训练集路径
        train_img = os.path.join(images_tr_path, filename)
        train_seg = os.path.join(labels_tr_path, filename)
        
        # 复制文件到训练集
        shutil.copy2(original_img, train_img)
        shutil.copy2(original_seg, train_seg)
        
        # 记录到CSV
        train_writer.writerow([original_img, original_seg, train_img, train_seg])

# 处理测试集
with open(test_csv, mode='w', newline='') as test_file:
    test_writer = csv.writer(test_file)
    test_writer.writerow(["Original MRI Path", "Original Seg Path", "New MRI Path", "New Seg Path"])
    
    for filename in test_files:
        # 定义原始路径
        original_img = os.path.join(images_full_path, filename)
        original_seg = os.path.join(labels_full_path, filename)
        
        # 定义测试集路径
        test_img = os.path.join(images_ts_path, filename)
        test_seg = os.path.join(labels_ts_path, filename)
        
        # 复制文件到测试集
        shutil.copy2(original_img, test_img)
        shutil.copy2(original_seg, test_seg)
        
        # 记录到CSV
        test_writer.writerow([original_img, original_seg, test_img, test_seg])

print("Data split into train/test sets completed and CSV logs created.")

Data split into train/test sets completed and CSV logs created.


In [10]:
import os
import json
from datetime import datetime

# Define the path to the dataset.json file
json_file_path = "../Processed/PII_Splite/dataset.json"  # Relative path from the ipython notebook

# Read the existing json file
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Update name, description, reference, license, and release date
data['name'] = "Placenta in Uterus"
data['description'] = "Segmentation of the placenta in the uterus"
data['reference'] = "Washington University in St. Louis"
data['licence'] = "null"
data['relase'] = datetime.now().strftime("%Y-%m-%d")

# Replace label 1 with "Placenta in Uterus"
data['labels']['1'] = "Placenta in Uterus"

# Define paths to the split directories
split_base_path = "../Processed/PII_Splite"
images_tr_path = os.path.join(split_base_path, "imagesTr")
labels_tr_path = os.path.join(split_base_path, "labelsTr")
images_ts_path = os.path.join(split_base_path, "imagesTs")
labels_ts_path = os.path.join(split_base_path, "labelsTs")

# Construct training and testing lists
training_list = []
testing_list = []

# Iterate over training set
for img_file in os.listdir(images_tr_path):
    label_file = img_file  # Assuming labels and images have the same filename
    training_list.append({
        "image": f"./imagesTr/{img_file}",
        "label": f"./labelsTr/{label_file}"
    })

# Iterate over testing set
for img_file in os.listdir(images_ts_path):
    testing_list.append(f"./imagesTs/{img_file}")

# Update the json fields for training and testing data
data['training'] = training_list
data['test'] = testing_list

# Update the number of training and testing samples
data['numTraining'] = len(training_list)
data['numTest'] = len(testing_list)

# Write the updated content back to the json file
with open(json_file_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("dataset.json updated successfully!")

dataset.json updated successfully!
